# Football predictions - Baseline models comparsion. 
XGBoost, CNN with Tab2Image transformation as proposed in this Nature paper - https://www.nature.com/articles/s41598-019-47765-6, and PyTorch implementation of TabNet proposed by https://github.com/dreamquark-ai/tabnet.

Based on a results of Premier League games from years 2016-2022.

In [1]:
# I'll be importing things on the go, as it will be much more clearer. First, let's update pip, otherwise the installation process od Soccerdata (https://pypi.org/project/soccerdata/) will take ages.
!pip install --upgrade pip
!pip install soccerdata # this lib I'm mentioning. It's just a scrapper from a couple of datasources, very handy. But SoFIFA scrapper there doesn't work well, as it's dropping a lot of players and data.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.0/986.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 13.8 MB/s eta 0:00

In [2]:
# some basic import that will help build a dataset for algorithms
import numpy as np
import pandas as pd
import soccerdata as sd
from datetime import datetime

[09/13/22 06:33:22] INFO     No custom team name replacements found. You can configure these in       ]8;id=595982;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=173755;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_config.py#87\87]8;;\
                             /root/soccerdata/config/teamname_replacements.json.                                   

INFO:root:No custom team name replacements found. You can configure these in /root/soccerdata/config/teamname_replacements.json.


                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=966240;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=15945;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_config.py#157\157]8;;\
                             /root/soccerdata/config/league_dict.json.                                             

INFO:root:No custom league dict found. You can configure additional leagues in /root/soccerdata/config/league_dict.json.


In [3]:
# First step is to get hands on the FBref database, as it appear the most interesting from feature generetion point of view.


# Build a dataset

We want to combine a couple of datasources provided by Soccerdata. I played with those data earlier before and udnerstand them pretty well, so let's build some helper functions to merge those datasources and produce some useful features, like rolling and accumulated stats of team form, goals and others useful components.

We have a couple of API's and we have a challenge to connect the data between them, as the team names vary here. So first we are about to build a step-by-step approach, and later combine everything in a function or class to build our database through a couple of seasons.

# FBref API 
It has at least three interesting API's. Fixture API, passing and shooting. From Passing and shooting API's we want to create a rolling mean dataframe of each team performance (clustered in "away_performance" and "home_performance" dataframes, so it would be easy to join it on game_id column as this parameter is used in each of the api's).

To do so, we will be doing following steps:

1. We will create a function for building a fixture of all the games from each season (And also a list of all team_names from this api call, as FBref vary internally)
2. Next, we will make passess and shots extractors, and combine them in a for loop iterator over a season dataframe, and it will return us a dataframe fror home and away performances for shooting and passess. We may consider this as a rolling performance (with a frame of size 5) of each teams clashing in the game. 
3. But to do so, we will probably create a translating function and we will use it upon the querying team names, cause they aren't matching.
4. After tests, and baseline training we'll consider adding a rolling xG_home and xG_away dataframe. It looks promising after a couple of games.

So let's get down to work.

In [65]:
### in our first step here we wanna clear dataframe from unnecesary informations.
def get_games_fbref(league_name:str, season:str):
    """Call Soccerdata FBref API and returns a dataframe object of a given params with some modifications:
    1. From 15 columns, we are keeping only date, home_team, away_team, home_xg, away_xg, game_id


    
    Example:
        get_games_fbref("ENG-Premier League","21")

        You can read docs about league names format in a documentation of Soccerdata:
        https://soccerdata.readthedocs.io/en/latest/reference/fbref.html#api-fbref

    Args:
        league_name: string
        season: string, usually just the last two numbers from and opening year of the season are fair enough.
        

    Returns:Dataframe
    """
    fbref = sd.FBref(leagues=league_name, seasons=season) # initialize class
    schedule = fbref.read_schedule().drop(["notes","attendance","venue",
                                           "week","day","time","score",
                                           "match_report","referee"],axis=1).dropna() #use a method to get data, a DataFrame already without NaN values. It should have a length of 380 games.

    return schedule, fbref

In [5]:
# xG rolling performance per team. fbref_schedule_21.loc[fbref_schedule_21.home_team == "Arsenal"][["home_xg","away_xg","game_id"]].rolling(5,0, closed="left").apply(np.mean).fillna(0)

In [6]:

fbref_schedule_21, epl21 = get_games_fbref(league_name="ENG-Premier League", season="21"); fbref_schedule_21.head(5)

                    INFO     Saving cached data to /root/soccerdata/data/FBref                        ]8;id=300435;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=554465;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /root/soccerdata/data/FBref


date  \
league             season game                                           
ENG-Premier League 2122   2021-08-13 Brentford-Arsenal      2021-08-13   
                          2021-08-14 Burnley-Brighton       2021-08-14   
                          2021-08-14 Chelsea-Crystal Palace 2021-08-14   
                          2021-08-14 Everton-Southampton    2021-08-14   
                          2021-08-14 Leicester City-Wolves  2021-08-14   

                                                                  home_team  \
league             season game                                                
ENG-Premier League 2122   2021-08-13 Brentford-Arsenal            Brentford   
                          2021-08-14 Burnley-Brighton               Burnley   
                          2021-08-14 Chelsea-Crystal Palace         Chelsea   
                          2021-08-14 Everton-Southampton            Everton   
                          2021-08-14 Leicester City-Wolves   Leicester City   

                                                             home_xg  away_xg  \
league             season game                                                  
ENG-Premier League 2122   2021-08-13 Brentford-Arsenal           1.3      1.4   
                          2021-08-14 Burnley-Brighton            1.0      1.3   
                          2021-08-14 Chelsea-Crystal Palace      0.9      0.3   
                          2021-08-14 Everton-Southampton         2.4      0.7   
                          2021-08-14 Leicester City-Wolves       0.8      1.1   

                                                                  away_team  \
league             season game                                                
ENG-Premier League 2122   2021-08-13 Brentford-Arsenal              Arsenal   
                          2021-08-14 Burnley-Brighton              Brighton   
                          2021-08-14 Chelsea-Crystal Palace  Crystal Palace   
                          2021-08-14 Everton-Southampton        Southampton   
                          2021-08-14 Leicester City-Wolves           Wolves   

                                                              game_id  
league             season game                                         
ENG-Premier League 2122   2021-08-13 Brentford-Arsenal       3adf2aa7  
                          2021-08-14 Burnley-Brighton        4eb36e37  
                          2021-08-14 Chelsea-Crystal Palace  6f454493  
                          2021-08-14 Everton-Southampton     c99ebbf5  
                          2021-08-14 Leicester City-Wolves   0b346a62

## From FBref we also have an API of player match stats and shots. 
It would be great to use them. Passess and shots are something calculated during the game, so the only information we can use is the accumulation and a rolling average of those params.

In [7]:
### LEt's start. with some helper function to extraxt mean of the passing performance per each team in the game. Later I'll add an option for picking mean or mode.

def get_pass_stats(game_id: str, team_name: str, fbref_class: sd.fbref.FBref) :

    """Calls soccerdata Fbref passing api about team_name passing performance in game.
    
    Args:
        game_id: string, game_id from FBref API
        team_name: team_name, from FBref_API
        fbref_class: class for a league/season pair.

    Returns: pd.core.series.Series
    """

    fbref = fbref_class # we want to pass the object of each season class

    match_stats = fbref.read_player_match_stats(stat_type="passing", match_id=game_id)
    match_stats.drop(["Nation","Pos","Age","Min","game_id"],inplace=True, axis=1)
    dropped = match_stats.droplevel(["league", "season","game"]).droplevel("player").fillna(0) # fill na with 0 as NaN appears while player had limited time on a pitch, so it's a better representation then mean or median or so.


    return dropped.groupby("team").mean().loc[team_name]

# and let's check if it works

get_pass_stats("6f454493", "Chelsea", epl21) # it works, so let's move on

[09/13/22 06:33:35] INFO     [3/1] Retrieving game with id=6f454493                                    ]8;id=728709;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=238377;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#501\501]8;;\

INFO:root:[3/1] Retrieving game with id=6f454493
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Long    Att          7.071429
        Cmp          5.571429
        Cmp%        58.528571
Medium  Att         20.000000
        Cmp         18.571429
        Cmp%        80.728571
Short   Att         21.142857
        Cmp         20.000000
        Cmp%        93.992857
Total   Att         50.357143
        Cmp         45.500000
        Cmp%        84.585714
        PrgDist    209.714286
        TotDist    838.000000
#                   14.714286
Ast                  0.071429
xA                   0.021429
KP                   0.428571
1/3                  4.071429
PPA                  1.285714
CrsPA                0.214286
Prog                 3.142857
Name: Chelsea, dtype: float64

In [66]:
fbref_type_of_stats = ["summary","keepers","passing","passing_types","defense","possession","misc"] # for now we will use only passing and shooting events. Shooting is in a different API.

In [9]:
### And here for the shots. Same approach
def get_team_shots(game_id:str, team_name:str, fbref_class: sd.fbref.FBref) -> pd.core.series.Series:

    """Calls soccerdata Fbref shooting api about team_name shooting performance in game.
    Group it in categories of shooting outcomes (woodwark, goals, blocked etc) and distance bins (10m, 20m and so)
    
    Args:
        game_id: string, game_id from FBref API
        team_name: team_name, from FBref_API
        fbref_class: class for a league/season pair.

    Returns: pd.core.series.Series
    """

    fbref = fbref_class

    shot_events = fbref.read_shot_events(game_id).reset_index().drop(["league","season","game","player"], axis=1) ; 
    outcomes = shot_events[shot_events.team == team_name].groupby(shot_events.outcome).size()
    bins = pd.cut(shot_events[shot_events.team == team_name].distance, [0,10,20,30,40,50,60,70,80,90,100], labels=["<10","<20","<30","<40","<50","<60","<70","<80","<90","<100"]) #clustering shots from distance into bins
    distance = shot_events[shot_events.team == team_name].groupby(bins).size()


    return pd.concat([outcomes,distance]) #pandas series

    
arsenal = get_team_shots("3adf2aa7", "Arsenal", epl21) # check if it works

[09/13/22 06:33:40] INFO     [1/1] Retrieving game with id=3adf2aa7                                    ]8;id=825645;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=383601;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#584\584]8;;\

INFO:root:[1/1] Retrieving game with id=3adf2aa7


In [10]:
### And we need to create team <-> game_id pairs

def get_games_id(fbref_dataframe: pd.core.frame.DataFrame, home=True):
    """Returns a map of team and game id for home or away games. IF home is true, returns home games. Else away games."""

    side = "home_team" if home else "away_team"
    games = {}
    for team in fbref_dataframe[side].unique():
        games[team] = fbref_dataframe.loc[fbref_dataframe[side] == team].game_id.values

    return games

In [11]:
def get_team_stats(side_dictionary: dict, passess, fbref_class: sd.fbref.FBref):
    """Collects from Soccerdata API based on a game_id and type of call.

    Args:
        side_dictionary: Dict, home_games or away_games,
        passess: Boolean; if True then uses get_pass_stats(), 
        else get_team_shots()

    Returns: Dictionary of team performances
    """

    ## FBref shots uses different team names then passess API... damn
    # if not passess:
    #     side_dictionary["West Ham"] = side_dictionary.pop("West Ham United")
    #     side_dictionary["Manchester Utd"] = side_dictionary.pop("Manchester United")
    #     side_dictionary["Brighton"] = side_dictionary.pop("Brighton & Hove Albion")
    #     side_dictionary["Wolves"] = side_dictionary.pop("Wolverhampton Wanderers")
    #     side_dictionary["Newcastle Utd"] = side_dictionary.pop("Newcastle United")
    #     side_dictionary[ "Tottenham"] = side_dictionary.pop("Tottenham Hotspur")


    dict_to_return = {}

    for key in side_dictionary:

        result = []

        if passess:   
            for game in side_dictionary[key]:    
                result.append(get_pass_stats(game,key,fbref_class)) 

        elif not passess: # simple switch for shooting function
            for game in side_dictionary[key]:    
                result.append(get_team_shots(game,key,fbref_class)) 

        dict_to_return[key] = result

    return dict_to_return

In [12]:
# We have to build a universal dictionary for translating team names between different API's.
# It will check the first X game ID's and query the specific api about the team's name. Collect it into np array, and overwrite existing keys in a passed dictionary.
def translate(schedule: pd.core.frame.DataFrame, fbref_class: sd.fbref.FBref, side_dict: dict, stats: str = "passing", window_size: int = 20):
    """
    returns translation of side dictionary, where all the keys are matching keys from the certain api

    """
    top20games = schedule.game_id[:window_size].values
    list_of_team_names =[]
    for game in top20games:
        list_of_team_names.append(fbref_class.read_player_match_stats(stat_type=stats, 
                                                                match_id=game, 
                                                                force_cache=False).droplevel(["league","season","game","player"]).index.unique().values)
        
    np_translate = np.unique(np.array(list_of_team_names).reshape(window_size*2,1)) # here we have a list of unqie team names

    for i in range(len(np_translate)):
        side_dict[np_translate[i]] = side_dict.pop(sorted(side_dict)[i]) # and those lines overwrite keys in a previously passed dictionary

    return side_dict

    

In [13]:
# And it's time for a rolling mean calculations

from scipy import stats # we will node stats.mode



def get_stats(stats_dict: dict, side_games_dict: dict):
    """Returns a dataframe of a rolling mean of stats"""
    dict = {}
    for key in stats_dict:

        results = pd.DataFrame(stats_dict[key]).rolling(5,0, closed="left").apply(np.mean).fillna(0)  #we are starting with zeros
        results.iloc[0] = stats.mode(results[1:], axis=0)[0] ; #but here we are filling the first rows with a mode per row.
        results["game_id"] = side_games_dict[key]
        dict[key] = results

    return pd.concat(dict)

In [14]:
def fbref_season(league_name: str, season: str) -> pd.core.frame.DataFrame:

    """
    Sequential call the FBref API for all of the interesting stats and compute some domain-based feature enginireeing on the output of API calls.

    Args:
        league_name: classmethod available_leagues() from documentation, string
        season: string, for example "21"

    Returns:
        a DataFrame of schedule
    """

    # Create a schedule and a class for a season
    schedule, fbref_class = get_games_fbref(league_name=league_name, season=season)


    # Create home_team and away_team to game_id links
    home_games = get_games_id(schedule)
    away_games = get_games_id(schedule, home=False)

    # Call for shot stats
    home_shots = get_team_stats(home_games, False, fbref_class)
    away_shots = get_team_stats(away_games, False, fbref_class)

    # Creates a DataFrame of rolling mean stats
    all_home_shots = get_stats(home_shots, home_games)
    all_away_shots = get_stats(away_shots, away_games)

    # Call for pass stats with dictionary translation
    passess_home_dict = translate(schedule, fbref_class, home_games)
    passess_away_dict = translate(schedule, fbref_class, away_games)

    home_passess = get_team_stats(passess_home_dict, True, fbref_class)
    away_passess = get_team_stats(passess_away_dict, True, fbref_class)

    #Creates a DataFrame of rolling mean of passess stats
    all_home_passess = get_stats(home_passess, home_games)
    all_away_passess = get_stats(away_passess, away_games)

    #Mergers on game_id

    schedule = schedule.merge(all_home_shots, on=["game_id"])
    schedule = schedule.merge(all_away_shots, on=["game_id"])
    schedule = schedule.merge(all_home_passess, on=["game_id"])
    schedule = schedule.merge(all_away_passess, on=["game_id"])

    return schedule

In [15]:
epl21_fbref_season = fbref_season(league_name="ENG-Premier League", season="21")

[09/13/22 06:33:46] INFO     Saving cached data to /root/soccerdata/data/FBref                        ]8;id=752065;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=906253;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /root/soccerdata/data/FBref


[09/13/22 06:33:47] INFO     [1/1] Retrieving game with id=3adf2aa7                                    ]8;id=758303;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=999209;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#584\584]8;;\

INFO:root:[1/1] Retrieving game with id=3adf2aa7
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


[09/13/22 06:33:48] INFO     [35/1] Retrieving game with id=19a03697                                   ]8;id=356879;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=230932;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#584\584]8;;\

...

INFO:root:[383/1] Retrieving game with id=99d0fee1


[09/13/22 07:18:05] INFO     [404/1] Retrieving game with id=7c8a79d2                                  ]8;id=372196;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=219299;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#501\501]8;;\

INFO:root:[404/1] Retrieving game with id=7c8a79d2


[09/13/22 07:18:06] INFO     [446/1] Retrieving game with id=8b0f3e28                                  ]8;id=174931;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py\fbref.py]8;;\:]8;id=773011;file:///usr/local/lib/python3.7/dist-packages/soccerdata/fbref.py#501\501]8;;\

INFO:root:[446/1] Retrieving game with id=8b0f3e28
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  validate=validate,


# Next step is API for ELO & MatchHistory.
We want to:

1. Create an API call for ELO; will use it later
2. Download season data from MatchHistory API. Create a list of team names, as those names are compatibile with ELO API.
time goals etc.
3. Clean DataFrame, and remain only a couple of columns
4. Return the dataframe.

## ELO API
The club ranks API

In [17]:
elo = sd.ClubElo() # initate the elo class, let's do it once and pass it to the function

def get_team_rank_by_day(team_name: str, date: datetime, elo_class=elo):
    """Ask the ELO api about the team rank at given time
    
    Example:
        get_team_rank_by_day("Chelsea","2022-10-8")

    Args:
        team_name: string
        date: datetime, %Y-%m-%d
        elo_class: Soccerdata ClubElo object

    Returns: Floating point number of team rank from ELO Api.
    """
    
    rank_by_date = elo_class.read_by_date(date).loc[team_name] # read_by_date method returns a dataframe, so we are using standard pandas filtering on top of it.
    return rank_by_date.elo # And we are substracting from Pandas series a floating point number.


# Let's check.
get_team_rank_by_day("Chelsea","2022-01-01")

[09/13/22 08:02:45] INFO     Saving cached data to /root/soccerdata/data/ClubElo                      ]8;id=751408;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=896656;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /root/soccerdata/data/ClubElo


1932.7019043

## MatchHistory API

Bookmakers odds and a game results

In [18]:
# Let's define a function that takes League name and a season year and returns formated. But let's start with a mapping function, it will help us label data.

def map_columns(factorized_object: tuple, sufix: str, use_sufix=False):
    """returns a dictionary that can be used for mapping and encoding classes that were vectorized/factorized
    
    Args:
        factorized_object: an output from pd.Series.factorize  
        sufix: string containing sufix to avoid columns name duplication
        duplicated: if there is occurance of duplication after applying column rename through the output of this function
    
    """
    dict_map = {}
    list_to_map = factorized_object[1].to_list()

    for a in range(len(list_to_map)):
        element = list_to_map[a] 
        index = list_to_map.index(element)
        dict_map[index] = element if not use_sufix else list_to_map[a] + sufix

    return dict_map


# I want to keep factorized values column in a dataset, that's why I exclude factorization from below helper function.
def one_hot_encode(factorized_object: tuple, data_series: pd.core.series.Series, use_sufix: False, sufix: str, ):

    """Returns a dataframe object of one hot encoding of categorical values in dataframe.
        If sufix is True then it uses it to map columns.

    Args:
        factorized_object: tuple, for mapping
        data_series: pandas series with categorical values. The same one to use during factorization!
        **sufix:  kwargs, a string
        use_sufix: Boolean

    Returns:
        A dataframe, pandas.core.frame.DataFrame
    """
    dummies = pd.get_dummies(data_series)
    dummies.rename(map_columns(factorized_object, sufix, use_sufix), axis=1, inplace=True)

    return dummies


# And the final league table function

def get_games_matchhistory(league_name:str, season:str):
    """Call Soccerdata MatchHistory and returns a dataframe object of a given params with some modifications:
    1. From 104 columns, we are keeping only date, home_team, away_team, FTR, FTHG, FTAG, HTHG, HTAR, HTR, AvgA, AvgD, AvgH
    2. We are vectorizing categorical (HTR, FTR) 

    
    Example:
        get_league_table("ENG-Premier League","21")

        You can read docs about league names format in a documentation of Soccerdata:
        https://soccerdata.readthedocs.io/en/latest/reference/matchhistory.html

    Args:
        league_name: string
        season: string, usually just the last two numbers from and opening year of the season are fair enough.
        

    Returns:Dataframe and a sorted np.array of team names
    """

    mh = sd.MatchHistory(leagues=league_name, seasons=season) #initialize a class object
    hist = mh.read_games() # and get actual dataframe
    hist = hist[["date","home_team","away_team",
                 "FTHG","FTAG","FTR","HTHG","HTAG","HTR",
                 "AvgH","AvgD","AvgA"
                 ]].reset_index().drop(["league","season","game"],axis=1)    # we are chaining some methods on a dataframe to get categories that we are interested in
                 

    ## Full Time Results factorization and one hot encoding merge

    factorized = hist.FTR.factorize() # using pandas to create factorize object for encoding H/D/A into 0,1,2
    hist.FTR = factorized[0] #turn FTR column into numerical one
    dummies_ftr = one_hot_encode(factorized, hist.FTR, use_sufix=True, sufix="_FT") # apply one hot encoding for Full Time Results
    hist = hist.join(dummies_ftr) # join tables

    ## Half Time Results factorization and one hot encoding merge
    
    factorized = hist.HTR.factorize() # using pandas to create factorize object for encoding H/D/A into 0,1,2
    hist.HTR = factorized[0] #turn FTR column into numerical one
    dummies_htr = one_hot_encode(factorized, hist.HTR, use_sufix=True, sufix="_HT") # apply one hot encoding for Full Time Results
    hist = hist.join(dummies_htr) # join tables
    
    hist["home_point"] = 0
    hist["draw_points"] = hist.D_FT
    hist["away_points"] = 0

    

    return hist

In [19]:
epl_mh_21 = get_games_matchhistory("ENG-Premier League","21")


[09/13/22 08:02:46] INFO     Saving cached data to /root/soccerdata/data/MatchHistory                 ]8;id=27687;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=28360;file:///usr/local/lib/python3.7/dist-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /root/soccerdata/data/MatchHistory


,date,home_team,away_team,FTHG,FTAG,FTR,HTHG,HTAG,HTR,AvgH,...,AvgA,H_FT,A_FT,D_FT,H_HT,A_HT,D_HT,home_point,draw_points,away_points
0,2021-01-11 20:00:00,Wolves,Everton,2,1,0,2,0,0,2.24,...,3.39,1,0,0,1,0,0,0,0,0
1,2021-01-12 20:15:00,Aston Villa,Man City,1,2,1,0,2,1,8.60,...,1.36,0,1,0,0,1,0,0,0,0
2,2021-01-12 20:15:00,Everton,Liverpool,1,4,1,1,2,1,7.52,...,1.41,0,1,0,0,1,0,0,0,0
3,2021-01-12 19:30:00,Southampton,Leicester,2,2,2,2,1,0,2.43,...,2.91,0,0,1,1,0,0,0,1,0
4,2021-01-12 19:30:00,Watford,Chelsea,1,2,1,1,1,2,10.30,...,1.31,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2022-12-02 15:00:00,Watford,Brighton,0,2,1,0,1,1,3.70,...,2.14,0,1,0,0,1,0,0,0,0
376,2022-12-03 15:00:00,Brentford,Burnley,2,0,0,0,0,2,2.20,...,3.65,1,0,0,0,0,1,0,0,0
377,2022-12-03 12:30:00,Brighton,Liverpool,0,2,1,0,1,1,6.20,...,1.53,0,1,0,0,1,0,0,0,0
378,2022-12-03 17:30:00,Man United,Tottenham,3,2,0,2,1,0,2.18,...,3.33,1,0,0,1,0,0,0,0,0


# Merge all of dataframes per one season
STEPS
1. Unify FBref and MatchHistory team names. We will overwrite Matchhistory names on FBref dataframe, as it will help us apply ELO rank on a whole dataframe.
2. Merge on team names.
3. Create new features from applying get_elo_rank for home and away teams.
4. Return a dataframe


In [20]:
def translate_for_elo(fbref_df: pd.core.frame.DataFrame, mh_df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """Returns a dictionary for a replacement for ELO API call """

    mh_df_names = np.sort(mh_df.home_team.unique())
    fbref_df_names = np.sort(fbref_df.home_team.unique())
    dict = {}
    for A,B in zip(fbref_df_names, mh_df_names):
        dict[A] = B

    return dict

# So we are only two features shy before we finish. So let's focus on a team form.

def get_team_form(team_name, dataframe):

    """Returns a Dataframe with rolling form"""

    team = dataframe.loc[(dataframe.home_team == team_name) | (dataframe.away_team == team_name)]
    team.home_point = team["home_team"] == team_name
    team.away_points = team["away_team"] == team_name

    team.away_points = team.away_points.astype(float)*3*team.A_FT
    team.home_poin = team.home_point.astype(float)*3*team.H_FT

    points_per_game = team[["away_points","home_point","draw_points"]].sum(axis=1)
    form = points_per_game.rolling(5,0, closed="left").apply(np.mean).fillna(0) #numpy mean creates 0/1 division in a first fixture so we have to fill na.

    return form

def create_df_with_teamform(df: pd.core.frame.DataFrame):
    """Fopr loop over a dataframe unique's team names. 
    After extracting all team names, create a form dataframe for each team
    Returns:
        Dataframe of all of the team forms
    """

    teams_form = {}
    for team in df.home_team.unique():
        team_form = get_team_form(team, df)
        teams_form[team] = team_form

    team_form = pd.DataFrame(teams_form)
    list_of_form = []
    for form in range(team_form.values.shape[0]):
        list_of_form.append((team_form.values[form][~np.isnan(team_form.values[form])]))

    return pd.DataFrame(list_of_form, columns=["home_form","away_form"])



def build_main(fbref_df: pd.core.frame.DataFrame, mh_df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """Returns a connected on home_team and away_team dataframe, with some new features like ELO rank and rolling means for a part of stats."""

    ### Unify names between dataframes
    main = fbref_df.replace(translate_for_elo(fbref_df,mh_df))


    ### Call ELO API and add a features of a team rank
    main["home_rank"] = main.apply(lambda row: get_team_rank_by_day(row["home_team"],row["date"]),axis=1)
    main["away_rank"] = main.apply(lambda row: get_team_rank_by_day(row["away_team"],row["date"]),axis=1)

    ## make a merge
    main = main.merge(mh_df, on=["home_team","away_team"])

    ## join an output of team form
    main = main.join(create_df_with_teamform(main))

    ## drop unnecessary

    main.drop(["H_FT","A_FT","D_FT","H_HT","A_FT","D_FT","H_HT","A_HT","home_xg","away_xg","FTHG","FTAG","HTHG","HTAG","HTR"], axis=1, inplace=True)
    
    return main




In [21]:
main = build_main(epl21_fbref_season,epl_mh_21) #it looks like everything works, so we can test this dataset on XGBoost

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [59]:
main.groupby("FTR").size()

FTR
0    163
1    129
2     88
dtype: int64

In [64]:
from scipy.stats import mode

mode(main[["AvgH","AvgA","AvgD"]])

ModeResult(mode=array([[1.16, 1.78, 3.37]]), count=array([[5, 4, 8]]))

# We also wanna cover some SoFifa Data


Let's take a look on FIFA game parameters. We want to substract home team mean skills from away team, and and the output as a feature.

Soccerdata also provides an API for SoFIFA scrapper, but there must be some data leakage there, so we will use following kaggle database.

You can download it from kaggle here - > https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset?rvi=1. I already downloaded it, so will just mount drive to get access.

In [37]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True) 

player_stats = pd.read_csv("/content/gdrive/MyDrive/AI_ML/so_fifa_stats/players_22.csv"); player_stats.shape # I have those files already downloaded, but You can easly do this through Kaggle API.

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(19239, 110)

In [38]:
params_to_drop = list(player_stats.columns)
premier_league_players = player_stats[player_stats.league_name == player_stats.league_name[4]]
plp_stats = premier_league_players.drop(params_to_drop[-32:-1]+['sofifa_id',
 'player_url','value_eur',
 'wage_eur',
 'league_level',
 'club_position',
 'dob',
 'club_loaned_from',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'real_face',
 'release_clause_eur',
 'league_name',
 'club_team_id',
 'short_name','nation_flag_url']	
, axis=1).set_index(["club_name","long_name"]).fillna(0)

In [39]:
#Now we have to temporarly store not numerical data and data that are not skills parameters in a different dataframe. Later on we will use them for the feature engineering if needed.
non_skills_stats = plp_stats[[ 'club_jersey_number','player_positions', 'age', 'height_cm',
       'weight_kg', 'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'player_tags',
       'player_traits']]
plp_stats.drop(['club_jersey_number','player_positions', 'age', 'height_cm',
       'weight_kg', 'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'player_tags',
       'player_traits'], axis=1, inplace=True)





In [40]:
# and now lasts of the helper functions - we want to add the difference of the skills between team as a features

def return_vectors(home_t, away_t, stats: pd.core.frame.DataFrame, home_handicap=False, handicap_size=1.05) -> tuple:

    home = (stats.loc[home_t].mean(axis=0).to_numpy()) if not home_handicap else torch.tensor(stats.loc[home_t].mean(axis=0).to_numpy()).float()*handicap_size
    away = (stats.loc[away_t].mean(axis=0).to_numpy())

    return home - away

def return_stacked(main_df: pd.core.frame.DataFrame):
    stacked_games = []
    for i in range(len(main_df)):
        stacked_games.append(return_vectors(main_df.iloc[i]["home_team"],main_df.iloc[i]["away_team"],plp_stats))

    return pd.DataFrame(np.array(stacked_games, dtype=np.float32)) # here we are building a stacked np array with all of the difference between teams, it should have 380 rows

In [41]:
fifa_teams = np.sort(plp_stats.droplevel("long_name").index.unique().to_numpy())

def translate_for_fifa(main_df: pd.core.frame.DataFrame, fifa_np: np.array):
    """Returns a dictionary for a replacement for ELO API call """

    main_df_names = np.sort(main_df.home_team.unique())
    dict = {}
    for A,B in zip(main_df_names, fifa_np):
        dict[A] = B

    return dict

In [42]:
main = main.replace(translate_for_fifa(main,fifa_teams))

In [43]:
fifa_params = return_stacked(main)

In [44]:
with_fifa = main.join(fifa_params) # at this point we have new dataframe with a bunch of Fifa features. Let's check if this helps somehow.

In [67]:
main.to_csv("/content/gdrive/MyDrive/Github/FootballPredictionsModel/data/main_raw.csv") # save Dframe to csv, to use in a model comparsion notebook.